In [3]:
#!pip install git+https://github.com/hassony2/torch_videovision

  Cloning https://github.com/hassony2/torch_videovision to /tmp/pip-req-build-cvio1xhh
  Running command git clone --filter=blob:none --quiet https://github.com/hassony2/torch_videovision /tmp/pip-req-build-cvio1xhh
  Resolved https://github.com/hassony2/torch_videovision to commit 785a810d1df2ccab6901fb98b38e358433e7b17a
  Preparing metadata (setup.py) ... done


In [5]:
#!pip install vidaug

In [42]:
from video_dataset import*
from transforms import *

In [44]:
root_dir = '/media/hdd2/astamoulakatos/PHD/datasets/archive/TrackingDataset/'
 
obj = os.scandir(root_dir)
 
# List all directories in the specified path
print("Files and Directories in '% s':" % root_dir)
class_paths = []
for entry in obj :
    if entry.is_dir():
        class_paths.append(entry.path)
        #print(entry.path)

Files and Directories in '/media/hdd2/astamoulakatos/PHD/datasets/archive/TrackingDataset/':


In [45]:
class_image_paths = []
end_idx = []
for c, class_path in enumerate(class_paths):
    for d in os.scandir(class_path):
        if d.is_dir:
            paths = sorted(glob.glob(os.path.join(d.path, '*.jpg')))
            # Add class idx to paths
            paths = [(p, c) for p in paths]
            class_image_paths.extend(paths)
            end_idx.extend([len(paths)])

end_idx = [0, *end_idx]
end_idx = torch.cumsum(torch.tensor(end_idx), 0)
seq_length = 16

sampler = MySampler(end_idx, seq_length)

tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

In [51]:
dataset = MyDataset(
        image_paths = class_image_paths,
        seq_length = seq_length,
        temp_transform = train_temp_transform,
        spat_transform = train_spat_transform,
        tensor_transform = tensor_transform,
        length = len(sampler),
        lstm = False,
        oned = False,
        augment = False,
        multi = 1)

In [50]:
loader = DataLoader(
        dataset,
        batch_size = 1,
        sampler = sampler,
        drop_last = True,
        num_workers = 0)  

In [49]:
for counter, data in enumerate(loader):
    print(data[0].shape)
    if counter==5:
        break

torch.Size([1, 3, 16, 480, 640])
torch.Size([1, 3, 16, 240, 320])
torch.Size([1, 3, 16, 480, 640])
torch.Size([1, 3, 16, 240, 320])
torch.Size([1, 3, 16, 480, 640])
torch.Size([1, 3, 16, 240, 320])
